# WIP: Back-test

This example is still WIP.
* Load Data
* Back Test - the Basic Idea

In [1]:
from grandma_stock_valuation import FileLogger, loadPacakgeData
from grandma_stock_valuation.back_test import GrandmaBackTester

# Refer to example_0_FileLogger.ipynb for details of the FileLogger.
logger = FileLogger()
logPrint = logger.logPandas

### Load Data

For this example, I will use the stored package data.

To query data from Yahoo, please refer to *example_1_yahoo_data_loader.ipynb*.

In [2]:
d_instrument_data, d_instrument = loadPacakgeData(verbose=2)

d_instrument

EEMA data contains 2530 rows, 2530 dates from 2012-02-09 to 2022-02-28.
IEV data contains 5431 rows, 5431 dates from 2000-07-28 to 2022-02-28.
IVV data contains 5479 rows, 5479 dates from 2000-05-19 to 2022-02-28.
VPL data contains 4273 rows, 4273 dates from 2005-03-10 to 2022-02-28.


{'IVV': 'SP500',
 'VPL': 'Developed Asia-Pacific',
 'IEV': 'Europe',
 'EEMA': 'Emerging Asia'}

In [3]:
#### TEMP ####
#l_select_inst = ['EEMA', 'VPL'] # , 'IVV', 
#d_instrument_data = {k:v for k,v in d_instrument_data.items() if k in l_select_inst}

In [4]:
logPrint("Keys of d_instrument_data:", str(d_instrument_data.keys()))

logPrint("IVV (SP500 ETF):", d_instrument_data['IVV'].head())

2022-03-14 07:15:59,663 INFO Keys of d_instrument_data: dict_keys(['EEMA', 'IEV', 'IVV', 'VPL'])
2022-03-14 07:15:59,668 INFO IVV (SP500 ETF): 
        date       open       high        low      close  close_adj   volume
0 2000-05-19  142.65625  142.65625  140.25000  140.68750  94.121216   775500
1 2000-05-22  140.59375  140.59375  136.81250  139.81250  93.535789  1850600
2 2000-05-23  140.21875  140.21875  137.68750  137.68750  92.114151   373900
3 2000-05-24  137.75000  140.06250  136.65625  139.75000  93.494003   400300
4 2000-05-25  140.03125  140.93750  137.87500  138.46875  92.636810    69600



### Back Test - the Basic Idea

WIP
* back test is to understand how your strategy behaves under various scenanios, not to prove your strategy can beat xxx.
* most investment products are not built to beat xxx.
* historical performance cannot represent future performance.


### Back Test - WIP

In [10]:
%load_ext line_profiler

import numpy as np
import pandas as pd

#backtester = GrandmaBackTester(
backtest_years=10
adjust_freq_months=3
init_parameters={'recent_months':0, 'train_years':10, 'min_train_years':5, 'date_end':None}
fit_parameters={'price_col':'close_adj', 'log':True, 'n_std':1.5}
valuate_parameters={'min_annual_return':0.01}
allocation_parameters={'transformation':'sigmoid', 'scale':1}
with_cash=False
with_correlation_weights=True
verbose=1
printfunc=logPrint
#)

_minimum_training_years = init_parameters['min_train_years']
_maximum_training_years = init_parameters['train_years']
_date_end = init_parameters['date_end']
_price_col = fit_parameters['price_col']

In [73]:
# clean price data, get start and end date of each ticker
d_instrument_prices = {}
d_instrument_start_dates = {}
d_instrument_end_dates = {}
for ticker, df_prices in d_instrument_data.items():
    df_prices = df_prices[df_prices[_price_col]>0][['date', _price_col]].reset_index(drop=True)
    df_prices['date'] = pd.to_datetime(df_prices['date'])
    d_instrument_prices[ticker] = df_prices
    d_instrument_start_dates[ticker] = df_prices['date'].min()
    d_instrument_end_dates[ticker] = df_prices['date'].max()

# dates of back-test periods
if _date_end is None:
    backtest_end_date = max(d_instrument_end_dates.values())
else:
    backtest_end_date = pd.to_datetime(_date_end)
backtest_end_date_ex = backtest_end_date.floor('D') + pd.DateOffset(days=1)

backtest_start_date = backtest_end_date_ex - pd.DateOffset(years=backtest_years)
if verbose > 0: printfunc(f"To backtest {backtest_years} years, from {backtest_start_date.date()} to {backtest_end_date.date()}")

at_least_start_date = backtest_start_date - pd.DateOffset(years=_minimum_training_years)
at_most_start_date = backtest_start_date - pd.DateOffset(years=_maximum_training_years)

for ticker, d in d_instrument_start_dates.items():
    if d > at_least_start_date:
        printfunc(f"{ticker}'s start date {d.date()} is beyond {at_least_start_date.date()} for full back-test.")

# combine cleaned price data of all instruments into one dateframe
def _selectPeriod(t_ticker_df):
    ticker, df_prices = t_ticker_df
    index_select = (df_prices['date']>=at_most_start_date) & (df_prices['date']<backtest_end_date_ex)
    df_prices = df_prices[index_select].set_index('date')
    df_prices.rename(columns={_price_col:'price_'+ticker}, inplace=True)
    return df_prices

df_instrument_prices = pd.concat(map(_selectPeriod, d_instrument_prices.items()), 
                                axis=1, join='outer', ignore_index=False, sort=False, copy=False)

df_instrument_prices.sort_index(ascending=True, inplace=True)
df_instrument_prices.fillna(method='pad', inplace=True)

index_start = df_instrument_prices.index.get_loc(backtest_start_date)

_d_instrument_start_dates = d_instrument_start_dates
_d_instrument_end_dates = d_instrument_end_dates
_backtest_start_date = backtest_start_date
_backtest_end_date_ex = backtest_end_date_ex
_index_start = index_start
_l_tickers = [s.replace('price_', '') for s in df_instrument_prices.columns]
_ar_dates = np.array(df_instrument_prices.index)
_ar_instrument_prices = df_instrument_prices.to_numpy()


2022-03-14 08:14:41,049 INFO To backtest 10 years, from 2012-03-01 to 2022-02-28
2022-03-14 08:14:41,050 INFO EEMA's start date 2012-02-09 is beyond 2007-03-01 for full back-test.
